## Multi Query
Breakdown complex or vague query into multiple simple ones.

In [1]:
from dotenv import load_dotenv
from langchain import hub
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter

import sys
import os
sys.path.insert(0, os.path.abspath('..'))

from utilities.file_utils import load_pdf_using_PyPDF
from utilities.text_utils import format_docs, log_chunks, get_unique_union_of_documents
from prompts import CHAT_HISTORY_PROMPT, CLARIFY_USER_QUERY_PROMPT, DIFFERENT_PERSPECTIVE_PROMPT

from docs.summary import BAJAJ_PDF_SUMMARY

In [3]:
file_path = "../docs/042f627c5d8f619cf62cc21f864b08dfd59059d0b9aab805d132e0014489d625.pdf"
pages = await load_pdf_using_PyPDF(file_path) # Average time with PyPDF: 2.5s

In [4]:
# Splitting into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(pages)

In [5]:
# Embeddings
# all_mini_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
bge_embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-small-en-v1.5")

embeddings = bge_embeddings

e:\Programming-Projects\Hackathon\hackrx6\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
# Creating Vector Store
vector_store = FAISS.from_documents(splits, embedding=embeddings)
# Store the vector DB locally to save processing time
vector_store.save_local("../vector_store/042f627c5d8f619cf62cc21f864b08dfd59059d0b9aab805d132e0014489d625")

In [7]:
# Load the saved vector store
vector_store = FAISS.load_local("../vector_store/042f627c5d8f619cf62cc21f864b08dfd59059d0b9aab805d132e0014489d625", embeddings, allow_dangerous_deserialization=True)

In [6]:
load_dotenv()

retriever = vector_store.as_retriever()

llm = ChatGoogleGenerativeAI(
    model="gemini-2.0-flash",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
)

In [ ]:
clarify_chain = CLARIFY_USER_QUERY_PROMPT | llm | StrOutputParser()

clarified_question = clarify_chain.invoke({
    "summary": BAJAJ_PDF_SUMMARY,
    "question": "46-year-old male, knee surgery in Pune, 3-month-old insurance policy"
})

print(clarified_question)

Okay, based on the document and the user's brief information, here's a more precise interpretation of the user's question:

"Given that I am a 46-year-old male with a 3-month-old Bajaj Allianz Global Health Care Policy (UIN: BAJHLIP23020V012223) and require knee surgery in Pune, am I covered for this surgery under my policy, considering any applicable waiting periods, sub-limits, or exclusions related to knee surgery or pre-existing conditions?  Specifically, I'd like to know:

*   Is knee surgery covered under the policy?
*   Are there any waiting periods that apply to knee surgery or related conditions, given that my policy is only 3 months old?
*   What are the potential sub-limits applicable to knee surgery?
*   Are there specific hospitals in Pune that are part of the network provider list for cashless claims?
*   Would pre-authorization be required for a cashless claim for this surgery?"


In [ ]:
# Query Decomposition
generate_queries = (
    DIFFERENT_PERSPECTIVE_PROMPT 
    | llm
    | StrOutputParser() 
    | (lambda x: x.split("\n"))
)

# Retrieve 
question = clarified_question   
retrieval_chain = generate_queries | retriever.map() | get_unique_union_of_documents
docs = retrieval_chain.invoke({"question": question})

e:\Programming-Projects\Hackathon\hackrx6\utilities\text_utils.py:20: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  return [loads(doc) for doc in unique_docs]


In [9]:
docs

[Document(id='e1021e6a-4c1d-4f69-ade4-87404cf5ed0f', metadata={'producer': 'Microsoft® Word 2016', 'creator': 'Microsoft® Word 2016', 'creationdate': '2022-06-16T20:06:13+05:30', 'author': 'Vinay Dhanokar/Head Office Pune/Corporate Communication/General', 'moddate': '2022-06-16T20:06:13+05:30', 'source': 'docs/BAJHLIP23020V012223.pdf', 'total_pages': 49, 'page': 45, 'page_label': '46'}, page_content='108  Neoadjuvant chemotherapy 307  Incision of foot fascia \n109  Adjuvant chemotherapy 308  calcaneum spur hydrocort injection \n110  Induction chemotherapy 309  Ganglion wrist hyalase injection \n111  Consolidation chemotherapy 310  Partial removal of metatarsal \n112  Maintenance chemotherapy 311  Repair / graft of foot tendon \n113  HDR Brachytherapy 312  Revision/Removal of Knee cap \nPlastic Surgery 313  Amputation follow-up surgery \n114  Construction skin pedicle flap  314  Exploration of ankle joint \n115  Gluteal pressure ulcer-Excision 315  Remove/graft leg bone lesion \n116  Mu

In [8]:
# Retrieve, Log and format the chunks
logged_retriever = retriever | RunnableLambda(log_chunks) | format_docs

In [9]:
# Add history using memory
session_histories = {}

def get_session_history(session_id: str):
    if session_id not in session_histories:
        session_histories[session_id] = InMemoryChatMessageHistory()
    return session_histories[session_id]

In [10]:
# Extract question and history to pass to retriever and prompt
def input_mapper(input):
    return {
        "context": logged_retriever.invoke(input["question"]),
        "question": input["question"],
        "history": input["history"]
    }

In [11]:
# Chain
rag_chain = (
    RunnableLambda(input_mapper)
    | CHAT_HISTORY_PROMPT
    | llm
    | StrOutputParser()
)

rag_with_history = RunnableWithMessageHistory(
    runnable=rag_chain,
    get_session_history=get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    output_messages_key="output"
)

In [ ]:
session_id = "user-1234" # Some unique identifier per user/session

# Question
# rag_with_history.invoke({"question": "what is the minimum and maximum In-patient Hospitalization Treatment Limits for Imperial plus plan?"}, 
#                         config={"configurable": {"session_id": session_id}})
rag_with_history.invoke({"question": "what was the last question i asked?"}, 
                        config={"configurable": {"session_id": session_id}})
# rag_with_history.invoke("46-year-old male, knee surgery in Pune, 3-month-old insurance policy")
# rag_with_history.invoke("knee surgery?")